<a href="https://colab.research.google.com/github/TrevorNgu/Number-Data-Model/blob/main/NumberDataV1M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip "./drive/MyDrive/NumberData/10000VV" -d "/content"

In [ ]:
import tensorflow as tf;
import tensorflow_datasets as tfds
import numpy as np
import PIL
import cv2
import os
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
from keras.preprocessing import image
from keras.applications.resnet50 import decode_predictions
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg19 import VGG19
from keras.models import Sequential
from keras.applications import resnet50
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense, Flatten
from keras import Model
from keras.models import load_model
from keras.utils import to_categorical
from keras import layers
from sklearn.model_selection import train_test_split
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

tf.debugging.experimental.enable_dump_debug_info(
    "/tmp/tfdbg2_logdir",
    tensor_debug_mode="FULL_HEALTH",
    circular_buffer_size=-1)

curr_model = None

def read_7_bytes(file):
  """Reads 7 bytes from a file and returns them as a NumPy array."""
  bytes = file.read(7)
  if len(bytes) < 7:
    return None
  return np.frombuffer(bytes, dtype=np.uint8)

def read_dir():
  path = "./train/"
  comb_arr = []
  for dire in os.listdir(path):
    fpath = os.path.join(path, dire)
    file = open(fpath, "rb")

    header = file.readline().decode("utf-8").strip()
    width = file.readline().decode("utf-8").strip()
    height = file.readline().decode("utf-8").strip()
    bitsperfilter = file.readline().decode("utf-8").strip()
    diameter = file.readline().decode("utf-8").strip()
    vertspace = file.readline().decode("utf-8").strip()
    horspace = file.readline().decode("utf-8").strip()
    horfirst = file.readline().decode("utf-8").strip()
    vertfirst = file.readline().decode("utf-8").strip()

    #print(header + " " + width + " " + height + " " + bitsperfilter + " " + diameter + " " + vertspace + " " + horspace + " " + horfirst + " " + vertfirst)

    array = np.array([])
    array.setflags(write=1)
    i = 0
    while True:
      i += 1
      bytes = read_7_bytes(file)
      bytes = np.copy(bytes)

      if bytes.all() == None:
        print(i)
        break

      bytes = bytes/255
      print(bytes)
      #for i in range(7):
        #bytes[i] = (bytes[i] - 128)
        #bytes[i] = (bytes[i] * 23000)

      #print(bytes)
      array = np.append(array, bytes)

    array = array.reshape(11, 9, 7)
    #print(array.shape)
    comb_arr.append(array)
    #val = array.reshape(11, 9, 7)
    #val = tf.convert_to_tensor(array, dtype="float32")
    #print("dir read")
  val = np.array(comb_arr)
  print(val.shape)
    #return val

def image_files(input_directory):
    filepaths=[]
    labels=[]

    digit_folders=os.listdir(input_directory)
    #print(digit_folders)

    for digit in digit_folders:
        path=os.path.join(input_directory, digit)
        flist=os.listdir(path)
        for f in flist:
            fpath=os.path.join(path,f)
            filepaths.append(fpath)
            labels.append(digit)
    return filepaths,labels

def load_images(filepaths):
    images = []
    for i in tqdm(range(len(filepaths))):
        img = None
        file = open(filepaths[i], "rb")
        if filepaths[i][-3:] != 'v1m':
          continue
        header = file.readline().decode("utf-8").strip()
        width = file.readline().decode("utf-8").strip()
        height = file.readline().decode("utf-8").strip()
        bitsperfilter = file.readline().decode("utf-8").strip()
        diameter = file.readline().decode("utf-8").strip()
        vertspace = file.readline().decode("utf-8").strip()
        horspace = file.readline().decode("utf-8").strip()
        horfirst = file.readline().decode("utf-8").strip()
        vertfirst = file.readline().decode("utf-8").strip()

        #print(header + " " + width + " " + height + " " + bitsperfilter + " " + diameter + " " + vertspace + " " + horspace + " " + horfirst + " " + vertfirst)

        array = np.array([])
        array.setflags(write=1)
        i = 0
        while True:
          i += 1
          bytes = read_7_bytes(file)
          bytes = np.copy(bytes)

          if bytes.all() == None:
            #print(i)
            break

          bytes = bytes/255
          if np.any(bytes <= 0) or np.any(bytes >= 1):
            print("found")

          #print(bytes)
          #for i in range(7):
            #bytes[i] = (bytes[i] - 128)
            #bytes[i] = (bytes[i] * 23000)

          #print(bytes)
          array = np.append(array, bytes)

        array = array.reshape(11, 9, 7)
        #img = image.load_img(filepaths[i], target_size=(32,32,3))
        #img = image.img_to_array(img)
        #img.astype('float32')
        #img = img/255
        images.append(array)

    images = np.array(images)
    return images

def createModel():
  global curr_model
  input_s = layers.Input((11,9,7))
  model_arch = tf.keras.applications.MobileNetV2(
  input_tensor=input_s,
  include_top=False,
  weights=None)

  curr_model = Sequential()
  curr_model.add(model_arch)
  curr_model.add(Flatten())
  curr_model.add(Dense(10, activation='softmax'))

  curr_model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

  curr_model.summary()

def loadModel():
  global curr_model
  curr_model = tf.keras.models.load_model('./drive/MyDrive/NumberData/model/my_model.h5')
  curr_model.summary()

def saveModel():
  global curr_model
  if curr_model == None:
    print("No model currently")
  else:
    print()
    #curr_model.save('./drive/MyDrive/NumberData/model')

def trainModel():
  directory = "./10000VV"
  filepaths, labels = image_files(directory)

  images = load_images(filepaths)
  x = tf.math.is_inf(images)
  flattened_tensor = tf.reshape(x, [-1])

  # Find the unique values
  unique_values, indices = tf.unique(flattened_tensor)

  # Print the unique values
  print(unique_values)
  y = to_categorical(labels, num_classes=10)
  x_train, x_test, y_train, y_test = train_test_split(images, y, random_state=42, test_size=0.2)

  print(np.any(np.isnan(x_train)))
  print(np.any(np.isnan(y_train)))
  print(np.any(np.isnan(x_test)))
  print(np.any(np.isnan(y_test)))

  print(np.any(np.isinf(x_train)))
  print(np.any(np.isinf(y_train)))
  print(np.any(np.isinf(x_test)))
  print(np.any(np.isinf(y_test)))

  print(np.any(np.isfinite(x_train)))
  print(np.any(np.isfinite(y_train)))
  print(np.any(np.isfinite(x_test)))
  print(np.any(np.isfinite(y_test)))

  print(np.any(x_train >= 1))
  print(np.any(x_train <= 0))
  print(np.any(y_train >= 11))
  print(np.any(y_train <= -1))
  print(np.any(x_test >= 1))
  print(np.any(x_test <= 0))
  print(np.any(y_test >= 11))
  print(np.any(y_test <= -1))


  curr_model.summary()

  history = curr_model.fit(x_train, y_train,
                           epochs=80,
                           batch_size=128,
                           validation_data=(x_test, y_test))
  curr_model.summary()

  curr_model.save('./drive/MyDrive/NumberData/model/my_model.h5')

def predict():
  global curr_model
  pathy = 'predict/'
  fname = input()
  pathy = pathy + fname
  file = open(pathy, "rb")

  header = file.readline().decode("utf-8").strip()
  width = file.readline().decode("utf-8").strip()
  height = file.readline().decode("utf-8").strip()
  bitsperfilter = file.readline().decode("utf-8").strip()
  diameter = file.readline().decode("utf-8").strip()
  vertspace = file.readline().decode("utf-8").strip()
  horspace = file.readline().decode("utf-8").strip()
  horfirst = file.readline().decode("utf-8").strip()
  vertfirst = file.readline().decode("utf-8").strip()

  #print(header + " " + width + " " + height + " " + bitsperfilter + " " + diameter + " " + vertspace + " " + horspace + " " + horfirst + " " + vertfirst)

  array = np.array([])
  array.setflags(write=1)
  i = 0
  while True:
    i += 1
    bytes = read_7_bytes(file)
    bytes = np.copy(bytes)

    if bytes.all() == None:
      print(i)
      break

    bytes = bytes/255
    print(bytes)
    #for i in range(7):
      #bytes[i] = (bytes[i] - 128)
      #bytes[i] = (bytes[i] * 23000)

    #print(bytes)
    array = np.append(array, bytes)

  array = array.reshape(-1, 11, 9, 7)
  print(array.shape)

  prediction = curr_model.predict(array)
  print(prediction)

while True:
  print("1. Create Model\n2. Load Model\n3. Save Model\n4. Train with Model\n5. Predict with Model\n6. Exit")
  x = int(input())

  match x:
    case 1:
        createModel()
    case 2:
        loadModel()
    case 3:
        saveModel()
    case 4:
        trainModel()
    case 5:
        predict()
    case 6:
        break

Num GPUs Available:  0
1. Create Model
2. Load Model
3. Save Model
4. Train with Model
5. Predict with Model
6. Exit
1


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_None (Fun  (None, 1, 1, 1280)        2259136   
 ctional)                                                        
                                                                 
 flatten_3 (Flatten)         (None, 1280)              0         
                                                                 
 dense_3 (Dense)             (None, 10)                12810     
                                                                 
Total params: 2271946 (8.67 MB)
Trainable params: 2237834 (8.54 MB)
Non-trainable params: 34112 (133.25 KB)
_________________________________________________________________
1. Create Model
2. Load Model
3. Save Model
4. Train with Model
5. Predict with Model
6. Exit
4


100%|██████████| 10000/10000 [00:39<00:00, 250.85it/s]


tf.Tensor([False], shape=(1,), dtype=bool)
False
False
False
False
False
False
False
False
True
True
True
True
False
False
False
False
False
False
False
False
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_None (Fun  (None, 1, 1, 1280)        2259136   
 ctional)                                                        
                                                                 
 flatten_3 (Flatten)         (None, 1280)              0         
                                                                 
 dense_3 (Dense)             (None, 10)                12810     
                                                                 
Total params: 2271946 (8.67 MB)
Trainable params: 2237834 (8.54 MB)
Non-trainable params: 34112 (133.25 KB)
_________________________________________________________________
Epoch 1/80
63/63 [==============================] -

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /tmp/tfdbg2_logdir